# Join 平台眾開講

## Setup

- [ ] Read messages from CSV.
- [ ] Cut message content by Jieba.
- [ ] Build embeddings using fastText word vectors pre-trained on Wikipedia corpus.
- [ ] Build training and validation datasets.

In [1]:
path = "data/join"

In [2]:
import pandas as pd, numpy as np
import jieba
import fasttext

ImportError: No module named jieba